In [1]:
import pyspark
from pyspark.sql import SparkSession
# sc = pyspark.SparkContext(master="spark://192.168.0.170:7077",
#                           appName='<app_name>')
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/17 13:14:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data Cleaning
data from https://data.boston.gov 's Blue Bike System Data
https://s3.amazonaws.com/hubway-data/index.html


In [2]:
df01 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202301-bluebikes-tripdata.csv")
df02 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202302-bluebikes-tripdata.csv")
df03 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202303-bluebikes-tripdata.csv")
df04 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202304-bluebikes-tripdata.csv")
df05 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202305-bluebikes-tripdata.csv")
df06 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202306-bluebikes-tripdata.csv")
df07 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202307-bluebikes-tripdata.csv")
df08 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202308-bluebikes-tripdata.csv")
df09 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202309-bluebikes-tripdata.csv")
df10 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202310-bluebikes-tripdata.csv")
df11 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202311-bluebikes-tripdata.csv")
df12 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202312-bluebikes-tripdata.csv")

In [3]:
def cleanDf(df):
    return df.withColumnRenamed("start station id","start_station_id").withColumnRenamed("end station id","end_station_id").withColumnRenamed("starttime","start_time") \
       .withColumnRenamed("start station name","start_station_name").withColumnRenamed("end station name","end_station_name").withColumnRenamed("stoptime","end_time")\
       .withColumnRenamed("start station latitude","start_lat").withColumnRenamed("end station latitude","end_lat")\
       .withColumnRenamed("start station longitude","start_lng").withColumnRenamed("end station longitude","end_lng")\
       .select('start_time',
                'end_time',
                'start_station_name',
                 'start_station_id',
                 'end_station_name',
                 'end_station_id',
                 'start_lat',
                 'start_lng',
                 'end_lat',
                 'end_lng')

def clean2(df):
   return df.withColumnRenamed("started_at","start_time").withColumnRenamed("ended_at","end_time").select('start_time',
                'end_time',
                'start_station_name',
                 'start_station_id',
                 'end_station_name',
                 'end_station_id',
                 'start_lat',
                 'start_lng',
                 'end_lat',
                 'end_lng')

In [4]:
df_num = cleanDf(df01).union(cleanDf(df02)).union(cleanDf(df03)).withColumnRenamed("start_station_id","start_id").withColumnRenamed("end_station_id","end_id").select("start_station_name","start_id").distinct()

In [5]:
df_str = clean2(df04).union(clean2(df05)).union(clean2(df06))\
        .union(clean2(df07)).union(clean2(df08)).union(clean2(df09))\
        .union(clean2(df10)).union(clean2(df11)).union(clean2(df12)).select("start_station_name","start_station_id").distinct()

In [6]:
df_id_str= df_str.select('start_station_name', 'start_station_id').withColumnRenamed("start_station_id","station_num").withColumnRenamed("start_station_name","station_name")
df_id_num= df_num.select('start_station_name', 'start_id').withColumnRenamed("start_id","station_id")


In [7]:
station_lookup_df = df_id_str.join(df_id_num, df_id_num['start_station_name'] ==df_id_str['station_name'], 'full').drop("start_station_name")

In [8]:
trips_01_df = cleanDf(df01).groupBy("start_station_id", "end_station_id").count()
trips_02_df = cleanDf(df02).groupBy("start_station_id", "end_station_id").count()
trips_03_df = cleanDf(df03).groupBy("start_station_id", "end_station_id").count()
trips_04_df = clean2(df04).groupBy("start_station_id", "end_station_id").count()
trips_05_df = clean2(df05).groupBy("start_station_id", "end_station_id").count()
trips_06_df = clean2(df06).groupBy("start_station_id", "end_station_id").count()
trips_07_df = clean2(df07).groupBy("start_station_id", "end_station_id").count()
trips_08_df = clean2(df08).groupBy("start_station_id", "end_station_id").count()
trips_09_df = clean2(df09).groupBy("start_station_id", "end_station_id").count()
trips_10_df = clean2(df10).groupBy("start_station_id", "end_station_id").count()
trips_11_df = clean2(df11).groupBy("start_station_id", "end_station_id").count()
trips_12_df = clean2(df12).groupBy("start_station_id", "end_station_id").count()

In [9]:
df01 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202303-bluebikes-tripdata.csv")
_df = cleanDf(df01)
start_done = _df.join(station_lookup_df, _df["start_station_name"] == station_lookup_df["station_name"], "outer").drop("start_station_id","station_name")\
    .withColumnRenamed("station_id","start_station_id").withColumnRenamed("station_num","start_station_num")
res_df = start_done.join(station_lookup_df, start_done["end_station_name"] == station_lookup_df["station_name"], "outer")
res_df.take(3)

[Row(start_time=datetime.datetime(2023, 3, 4, 9, 16, 2, 910000), end_time=datetime.datetime(2023, 3, 4, 9, 49, 10, 160000), start_station_name='175 N Harvard St', end_station_name='175 N Harvard St', end_station_id=149, start_lat=42.363796, start_lng=-71.129164, end_lat=42.363796, end_lng=-71.129164, start_station_num='A32019', start_station_id=149, station_name='175 N Harvard St', station_num='A32019', station_id=149),
 Row(start_time=datetime.datetime(2023, 3, 10, 18, 15, 8, 719000), end_time=datetime.datetime(2023, 3, 10, 19, 3, 0, 666000), start_station_name='175 N Harvard St', end_station_name='175 N Harvard St', end_station_id=149, start_lat=42.363796, start_lng=-71.129164, end_lat=42.363796, end_lng=-71.129164, start_station_num='A32019', start_station_id=149, station_name='175 N Harvard St', station_num='A32019', station_id=149),
 Row(start_time=datetime.datetime(2023, 3, 10, 18, 15, 46, 263000), end_time=datetime.datetime(2023, 3, 10, 19, 3, 9, 408000), start_station_name='175

In [10]:
def addStationNum(_df):
    _df = cleanDf(_df)
    return _df.join(station_lookup_df, _df["start_station_id"] == station_lookup_df["station_id"], "full_outer").drop("start_station_id")\
    .withColumnRenamed("station_id","start_station_id").withColumnRenamed("station_id","start_station_id")

In [15]:
df_01 = addStationNum(df01)
df_01.groupBy("start_station_id", "end_station_id").count().take(1)

[Row(start_station_id=3, end_station_id=22, count=8)]

In [11]:
trips_10_df.sort("count", ascending=False).take(10)

[Row(start_station_id='M32006', end_station_id='B32016', count=941),
 Row(start_station_id='B32016', end_station_id='M32006', count=876),
 Row(start_station_id='M32006', end_station_id='M32042', count=619),
 Row(start_station_id='M32018', end_station_id='M32024', count=616),
 Row(start_station_id='M32042', end_station_id='M32005', count=528),
 Row(start_station_id='M32042', end_station_id='M32037', count=495),
 Row(start_station_id='M32042', end_station_id='M32006', count=490),
 Row(start_station_id='M32011', end_station_id='M32041', count=488),
 Row(start_station_id='M32006', end_station_id='M32011', count=458),
 Row(start_station_id='A32002', end_station_id='B32060', count=450)]

In [10]:
from pyspark.sql.functions import col, when
# Join the two DataFrames based on 'station_name'


joined_df = df_num.join(df_station_id, 
                        df_num['start_station_name'] == df_station_id['station_name'], 
                        'left_outer')

# Replace 'start_id' with 'station_id' when there's a match, otherwise keep the original 'start_id'
result_df = joined_df.withColumn('start_id', when(col('station_id').isNotNull(), col('station_id')).otherwise(col('start_id')))\

# Drop the additional 'station_id' column if needed
df2 = result_df.drop('station_id').drop('station_name')

#Redo for end
joined_df = df2.join(df_station_id, 
                        df2['end_station_name'] == df_station_id['station_name'], 
                        'left_outer')

# Replace 'start_id' with 'station_id' when there's a match, otherwise keep the original 'start_id'
result_df = joined_df.withColumn('end_id', when(col('station_id').isNotNull(), col('station_id')).otherwise(col('end_id')))\

# Drop the additional 'station_id' column if needed
result_df = result_df.drop('station_id').drop('station_name')
df2023 = result_df.union(df_str.withColumnRenamed('end_station_id', 'end_id').withColumnRenamed('start_station_id', 'start_id'))

In [7]:
df2023.take(1)

[Row(start_time=datetime.datetime(2023, 1, 1, 0, 2, 54, 80000), end_time=datetime.datetime(2023, 1, 1, 0, 7, 4, 410000), start_station_name='Copley Square - Dartmouth St at Boylston St', start_id='D32005', end_station_name='Prudential Center - 101 Huntington Ave', end_id='C32007', start_lat=42.34992828230057, start_lng=-71.07739206866427, end_lat=42.34652003998412, end_lng=-71.08065776545118)]

In [11]:
result_df.groupBy("start_id", "end_id").count().collect()

24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:51:39 WARN RowBasedKeyValueBatch: Calling spill() on

[2283.838s][warning][gc,alloc] Executor task launch worker for task 0.0 in stage 32.0 (TID 212): Retried waiting for GCLocker too often allocating 131074 words


24/01/10 12:52:47 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 0.0 in stage 32.0 (TID 212),5,main]
java.lang.OutOfMemoryError: Java heap space
24/01/10 12:52:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/10 12:52:47 WARN TaskSetManager: Lost task 0.0 in stage 32.0 (TID 212) (192.168.0.170 executor driver): java.lang.OutOfMemoryError: Java heap space

24/01/10 12:52:47 ERROR TaskSetManager: Task 0 in stage 32.0 failed 1 times; aborting job
24/01/10 12:52:47 WARN TaskSetManager: Lost task 4.0 in stage 32.0 (TID 216) (192.168.0.170 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 212) (192.168.0.170 executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:)


Py4JJavaError: An error occurred while calling o413.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 212) (192.168.0.170 executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2844)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2780)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2779)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2779)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(D

In [ ]:
df_str.groupBy("start_id", "end_id").count().collect()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/harry/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/harry/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/harry/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
df.groupBy("start_station_id", "end_station_id").count().filter(df.start_station_id == 91).sort("count", ascending=False).collect()

In [ ]:
df.groupBy("start_station_id", "end_station_id").count().sort("count", ascending=False).collect()

In [ ]:
df1 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202301-bluebikes-tripdata.csv").select("start station name", "end station name", "starttime").withColumnRenamed("start station name","start_station_name").withColumnRenamed("end station name","end_station_name").withColumnRenamed("starttime","startTime")
df2 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202302-bluebikes-tripdata.csv").select("start station name", "end station name", "starttime").withColumnRenamed("start station name","start_station_name").withColumnRenamed("end station name","end_station_name").withColumnRenamed("starttime","startTime")
df3 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202303-bluebikes-tripdata.csv").select("start station name", "end station name", "starttime").withColumnRenamed("start station name","start_station_name").withColumnRenamed("end station name","end_station_name").withColumnRenamed("starttime","startTime")
df4 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202304-bluebikes-tripdata.csv").select("start_station_name", "end_station_name", "started_at").withColumnRenamed("started_at","startTime")
df5 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202305-bluebikes-tripdata.csv").select("start_station_name", "end_station_name", "started_at").withColumnRenamed("started_at","startTime")
df6 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202306-bluebikes-tripdata.csv").select("start_station_name", "end_station_name", "started_at").withColumnRenamed("started_at","startTime")
df7 = spark.read.options(header='True',inferSchema='True',delimiter=',').csv("./202307-bluebikes-tripdata.csv").select("start_station_name", "end_station_name", "started_at").withColumnRenamed("started_at","startTime")

df = df1.union(df2).union(df3).union(df4).union(df5).union(df6).union(df7)
df.take(1)



In [ ]:
df.groupBy("start_station_name", "end_station_name").count().sort("count", ascending=False).collect()